In [66]:
import tensorflow as tf
import numpy as np
from tensorflow import keras
import os
import cv2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt
import keras
from tensorflow.keras.optimizers import Adam

In [67]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [68]:
train_dir = "/Users/jarvis/pymycod/tensorflow_AI/planetary/satellite/train"
test_dir = "/Users/jarvis/pymycod/tensorflow_AI/planetary/satellite/test"
val_dir = "/Users/jarvis/pymycod/tensorflow_AI/planetary/satellite/valid"

train_dataset = ImageDataGenerator(rescale=1.0/255,
                                  rotation_range=0.4,
                                  height_shift_range=0.2, 
                                  zoom_range=0.2, 
                            )
train_gen = train_dataset.flow_from_directory(
    train_dir,
    target_size=(256,256),
    batch_size=128,
    class_mode='binary'
)

test_dataset = ImageDataGenerator(rescale=1.0/255,
                                  rotation_range=0.4,
                                  height_shift_range=0.2, 
                                  zoom_range=0.2, 
                            )
test_gen = test_dataset.flow_from_directory(
    test_dir,
    target_size=(256,256),
    batch_size=128,
    class_mode='binary'
)
val_dataset = ImageDataGenerator(rescale=1.0/255,
                                  rotation_range=0.4,
                                  height_shift_range=0.2, 
                                  zoom_range=0.2, 
                            )
val_gen = val_dataset.flow_from_directory(
    val_dir,
    target_size=(256,256),
    batch_size=128,
    class_mode='binary'
)

Found 30250 images belonging to 2 classes.
Found 6300 images belonging to 2 classes.
Found 6300 images belonging to 2 classes.


In [69]:
steps_train = 30250//128
steps_train

236

In [70]:
test_gen.class_indices

{'nowildfire': 0, 'wildfire': 1}

In [71]:
model = keras.Sequential([keras.layers.Conv2D(32,(3,3),activation='relu',input_shape=(256,256,3)),
                               keras.layers.MaxPool2D(2,2),
                               keras.layers.Conv2D(64,(3,3),activation='relu'),
                               keras.layers.MaxPool2D(2,2),
                               keras.layers.BatchNormalization(),
                               keras.layers.Dropout(0.2),
                               keras.layers.Conv2D(128,(3,3),activation='relu'),
                               keras.layers.MaxPool2D(2,2),
                               keras.layers.Dropout(0.2),
                               keras.layers.Conv2D(128,(3,3),activation='relu'),
                               keras.layers.MaxPool2D(2,2),
                               keras.layers.BatchNormalization(),
                               keras.layers.Flatten(),
                               keras.layers.Dense(512,activation='relu'),
                               keras.layers.Dense(64,activation='relu'),
                               keras.layers.Dense(1,activation='sigmoid')
                               ])


model.compile(loss = 'binary_crossentropy',
                optimizer = Adam(learning_rate=0.001),
                metrics=['accuracy'])


In [72]:
model.summary()

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_40 (Conv2D)          (None, 254, 254, 32)      896       
                                                                 
 max_pooling2d_40 (MaxPoolin  (None, 127, 127, 32)     0         
 g2D)                                                            
                                                                 
 conv2d_41 (Conv2D)          (None, 125, 125, 64)      18496     
                                                                 
 max_pooling2d_41 (MaxPoolin  (None, 62, 62, 64)       0         
 g2D)                                                            
                                                                 
 batch_normalization_6 (Batc  (None, 62, 62, 64)       256       
 hNormalization)                                                 
                                                     

In [73]:
from keras.callbacks import ModelCheckpoint
model_path = "/Users/jarvis/pymycod/tensorflow_AI/trained_models/lr_model_satellite_wildfires.h5"
checkpoint1 = ModelCheckpoint(model_path,monitor="val_accuracy",verbose=1,save_best_only=True,mode=max)
callbacks=[checkpoint1]

In [74]:
history = model.fit(train_gen,epochs=100,verbose=2,batch_size=128,steps_per_epoch=steps_train,validation_data=test_gen,callbacks=callbacks)

Epoch 1/100


2023-04-21 12:30:04.244869: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


KeyboardInterrupt: 